In [1]:
# Setup
import numpy as np
import pandas as pd
import pyarrow as pa
import datetime

import pykx as kx
kx.q.system.console_size = [15, 100]



In [2]:
tab_BTC = kx.q.read.csv('data/BTCUSDT.csv')

# Create a new column in tab_BTC listing the date for each row
tab_BTC['date'] = tab_BTC['timestamp'].date

# Section 4: Querying Tables

## qSQL

### select()

In [3]:
# Find all dates where the closing price is greater than 73000 
tab_BTC.select(where=kx.Column('close') >73000, by=kx.Column('date'))

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
date,,,,,,,,,,,,
2024.03.13,2024.03.13D23:59:00.000000000,73039.87e,73072.41e,73039.86e,73072.41e,20.26091,2024.03.13D23:59:59.999000000,1480114f,1264h,17.82432,1302127f,0b
2024.03.14,2024.03.14D12:11:00.000000000,73011e,73011.01e,73011e,73011.01e,12.11509,2024.03.14D12:11:59.999000000,884534.9,480h,2.9191,213126.4,0b


In [4]:
# What is the average closing price
tab_BTC.select(columns=kx.Column('close').avg().name('average_Close'))

,average_Close
,
0,23242.17


##### Exercise 12
- Return the total number of transactions where volume is zero

In [5]:
tab_BTC.select(columns=kx.Column('i').count().name('transactions'), where=kx.Column('volume') == 0.0)

,transactions
,
0,23863


### exec()

In [6]:
# Select the last item of the table
tab_BTC.exec()

,
,
timestamp,2024.06.03D03:00:00.000000000
open,68416.09e
high,68451.07e
low,68412e
close,68431.45e
volume,18.64174
close_time,2024.06.03D03:00:59.999000000
quote_asset_volume,1275749f
number_of_trades,974h


In [7]:
tab_BTC.exec(kx.Column('close').last(), by = kx.Column('date'), where = kx.Column('ignore') == False)

,
,
2017.08.17,4285.08e
2017.08.18,4108.37e
2017.08.19,4139.98e
2017.08.20,4086.29e
2017.08.21,4016e
2017.08.22,4040e
2017.08.23,4114.01e
2017.08.24,4316.01e
2017.08.25,4280.68e


### update()

In [8]:
# Update Ignore to true for rows where closing price is less than 5000
tab_BTC.update({'ignore':True}, where=kx.Column('close') < 5000.0)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,date
,,,,,,,,,,,,,
0,2017.08.17D04:00:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,1.775183,2017.08.17D04:00:59.999000000,7564.907,3h,0.075183,320.3909,1b,2017.08.17
1,2017.08.17D04:01:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0f,2017.08.17D04:01:59.999000000,0f,0h,0f,0f,1b,2017.08.17
2,2017.08.17D04:02:00.000000000,4280.56e,4280.56e,4280.56e,4280.56e,0.261074,2017.08.17D04:02:59.999000000,1117.543,2h,0.261074,1117.543,1b,2017.08.17
3,2017.08.17D04:03:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0.012008,2017.08.17D04:03:59.999000000,51.17185,3h,0.012008,51.17185,1b,2017.08.17
4,2017.08.17D04:04:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0.140796,2017.08.17D04:04:59.999000000,599.9993,1h,0.140796,599.9993,1b,2017.08.17
5,2017.08.17D04:05:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0f,2017.08.17D04:05:59.999000000,0f,0h,0f,0f,1b,2017.08.17
6,2017.08.17D04:06:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0f,2017.08.17D04:06:59.999000000,0f,0h,0f,0f,1b,2017.08.17
7,2017.08.17D04:07:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0f,2017.08.17D04:07:59.999000000,0f,0h,0f,0f,1b,2017.08.17
8,2017.08.17D04:08:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0f,2017.08.17D04:08:59.999000000,0f,0h,0f,0f,1b,2017.08.17


### delete()

In [9]:
tab_BTC['test_column'] = "TEST!"
tab_BTC.head()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,date,test_column
,,,,,,,,,,,,,,
0,2017.08.17D04:00:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,1.775183,2017.08.17D04:00:59.999000000,7564.907,3h,0.075183,320.3909,0b,2017.08.17,TEST!
1,2017.08.17D04:01:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0f,2017.08.17D04:01:59.999000000,0f,0h,0f,0f,0b,2017.08.17,TEST!
2,2017.08.17D04:02:00.000000000,4280.56e,4280.56e,4280.56e,4280.56e,0.261074,2017.08.17D04:02:59.999000000,1117.543,2h,0.261074,1117.543,0b,2017.08.17,TEST!
3,2017.08.17D04:03:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0.012008,2017.08.17D04:03:59.999000000,51.17185,3h,0.012008,51.17185,0b,2017.08.17,TEST!
4,2017.08.17D04:04:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0.140796,2017.08.17D04:04:59.999000000,599.9993,1h,0.140796,599.9993,0b,2017.08.17,TEST!


In [10]:
# Delete a column from the table where the deletion is persisted (inplace=True)
tab_BTC.delete(kx.Column('test_column'), inplace=True)
tab_BTC.head()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,date,test_column
,,,,,,,,,,,,,,
0,2017.08.17D04:00:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,1.775183,2017.08.17D04:00:59.999000000,7564.907,3h,0.075183,320.3909,0b,2017.08.17,TEST!
1,2017.08.17D04:01:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0f,2017.08.17D04:01:59.999000000,0f,0h,0f,0f,0b,2017.08.17,TEST!
2,2017.08.17D04:02:00.000000000,4280.56e,4280.56e,4280.56e,4280.56e,0.261074,2017.08.17D04:02:59.999000000,1117.543,2h,0.261074,1117.543,0b,2017.08.17,TEST!
3,2017.08.17D04:03:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0.012008,2017.08.17D04:03:59.999000000,51.17185,3h,0.012008,51.17185,0b,2017.08.17,TEST!
4,2017.08.17D04:04:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0.140796,2017.08.17D04:04:59.999000000,599.9993,1h,0.140796,599.9993,0b,2017.08.17,TEST!


In [11]:
tab_BTC.delete(where=kx.Column('close') < 5000)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,date,test_column
,,,,,,,,,,,,,,
0,2017.10.12D07:58:00.000000000,4988e,5009.93e,4988e,5009.93e,8.58558,2017.10.12D07:58:59.999000000,42912.09,34h,8.533794,42652.65,0b,2017.10.12,TEST!
1,2017.10.12D07:59:00.000000000,5010.2e,5010.2e,5009e,5010.2e,1.532306,2017.10.12D07:59:59.999000000,7676.94,8h,0.984092,4930.426,0b,2017.10.12,TEST!
2,2017.10.12D08:00:00.000000000,5010.2e,5031.35e,5010.2e,5031.35e,3.037492,2017.10.12D08:00:59.999000000,15219.53,16h,3.03566,15210.32,0b,2017.10.12,TEST!
3,2017.10.12D08:01:00.000000000,5040e,5050e,5031.57e,5031.57e,0.691529,2017.10.12D08:01:59.999000000,3483.883,4h,0.43,2167.982,0b,2017.10.12,TEST!
4,2017.10.12D08:02:00.000000000,5050e,5079e,5050e,5079e,2.475541,2017.10.12D08:02:59.999000000,12532.05,26h,2.475541,12532.05,0b,2017.10.12,TEST!
5,2017.10.12D08:03:00.000000000,5079e,5099e,5079e,5099e,0.430276,2017.10.12D08:03:59.999000000,2188.582,7h,0.298516,1519.373,0b,2017.10.12,TEST!
6,2017.10.12D08:04:00.000000000,5099e,5142.34e,5099e,5110.29e,5.157748,2017.10.12D08:04:59.999000000,26360.63,27h,5.034348,25730.26,0b,2017.10.12,TEST!
7,2017.10.12D08:05:00.000000000,5110.29e,5110.29e,5080.01e,5105.26e,4.525278,2017.10.12D08:05:59.999000000,23094.61,24h,2.748762,14031.81,0b,2017.10.12,TEST!
8,2017.10.12D08:06:00.000000000,5080e,5105.26e,5064.77e,5080e,2.73163,2017.10.12D08:06:59.999000000,13919.73,22h,1.326769,6758.605,0b,2017.10.12,TEST!


## Standard SQL Interface

In [12]:
kx.q.sql('select * from $1', tab_BTC)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,date,test_column
,,,,,,,,,,,,,,
0,2017.08.17D04:00:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,1.775183,2017.08.17D04:00:59.999000000,7564.907,3h,0.075183,320.3909,0b,2017.08.17,TEST!
1,2017.08.17D04:01:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0f,2017.08.17D04:01:59.999000000,0f,0h,0f,0f,0b,2017.08.17,TEST!
2,2017.08.17D04:02:00.000000000,4280.56e,4280.56e,4280.56e,4280.56e,0.261074,2017.08.17D04:02:59.999000000,1117.543,2h,0.261074,1117.543,0b,2017.08.17,TEST!
3,2017.08.17D04:03:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0.012008,2017.08.17D04:03:59.999000000,51.17185,3h,0.012008,51.17185,0b,2017.08.17,TEST!
4,2017.08.17D04:04:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0.140796,2017.08.17D04:04:59.999000000,599.9993,1h,0.140796,599.9993,0b,2017.08.17,TEST!
5,2017.08.17D04:05:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0f,2017.08.17D04:05:59.999000000,0f,0h,0f,0f,0b,2017.08.17,TEST!
6,2017.08.17D04:06:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0f,2017.08.17D04:06:59.999000000,0f,0h,0f,0f,0b,2017.08.17,TEST!
7,2017.08.17D04:07:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0f,2017.08.17D04:07:59.999000000,0f,0h,0f,0f,0b,2017.08.17,TEST!
8,2017.08.17D04:08:00.000000000,4261.48e,4261.48e,4261.48e,4261.48e,0f,2017.08.17D04:08:59.999000000,0f,0h,0f,0f,0b,2017.08.17,TEST!


##### Exercise 13
- Using the Standard SQL Interface, return the average number of trades by date

In [13]:
kx.q.sql('select date, avg(number_of_trades) from $1 group by date', tab_BTC)

,date,number_of_trades
,,
0,2017.08.17,2.855833
1,2017.08.18,3.634028
2,2017.08.19,1.495139
3,2017.08.20,1.611806
4,2017.08.21,2.758333
5,2017.08.22,4.509722
6,2017.08.23,5.992361
7,2017.08.24,4.338194
8,2017.08.25,4.551389


## IPC

In [14]:
import subprocess
import time

try:
    with kx.PyKXReimport():
        proc = subprocess.Popen(
            ('q', '-p', '5000')
        )
    time.sleep(2)
except:
    raise kx.QError('Unable to create q process on port 5000')


In [15]:
conn = kx.SyncQConnection(port = 5000)

In [16]:
conn('tab:([]col1:100?`a`b`c;col2:100?1f;col3:100?0Ng)')
conn('select from tab where col1=`a')

,col1,col2,col3
,,,
0,a,0.01974141,ddb87915-b672-2c32-a6cf-296061671e9d
1,a,0.5611439,580d8c87-e557-0db1-3a19-cb3a44d623b1
2,a,0.8685452,2d948578-e9d6-79a2-8207-9df7a71f0b3b
3,a,0.3460797,cddeceef-9ee9-3847-9172-3e3d7ab39b26
4,a,0.5046331,1c22a468-9492-2173-9e4f-9003a23d02b7
5,a,0.765905,5e9cd21b-88c5-bbf5-7215-6409e115a2a4
6,a,0.8794685,3462beab-42ee-ccad-989b-8d69f070dffc
7,a,0.02487862,bc150163-c551-0eba-8871-9767f5c0e3d5
8,a,0.3664924,dd6b4a2b-c046-e464-a0b9-efb96ed5f0eb


In [17]:
conn.qsql.select('tab', where=(kx.Column('col1') == 'a') & (kx.Column('col2') < 0.3))

,col1,col2,col3
,,,
0,a,0.01974141,ddb87915-b672-2c32-a6cf-296061671e9d
1,a,0.02487862,bc150163-c551-0eba-8871-9767f5c0e3d5
2,a,0.2073435,ee853957-d502-d30d-5945-bf8c97022332
3,a,0.2188574,d9a3e171-b1cf-0271-507a-0fba0b52e6ff
4,a,0.1451855,ea4d0269-375c-d73b-96f0-6bb6334ca423
5,a,0.1497004,1cce6bdd-e34b-ba4f-8c01-31d098d81221
6,a,0.166486,6417d4b3-3fc6-e35a-1c34-8c5c3327b1e8
7,a,0.2643322,f294c3cb-a6da-e15d-c8e0-3a848d2abf10
8,a,0.07841939,020715aa-8ffa-e1d3-9c68-3ad7919d4f5e


In [18]:
conn('\l s.k_')
conn.sql('SELECT * FROM tab where col2>=0.5')

,col1,col2,col3
,,,
0,a,0.5611439,580d8c87-e557-0db1-3a19-cb3a44d623b1
1,a,0.8685452,2d948578-e9d6-79a2-8207-9df7a71f0b3b
2,b,0.7716917,52cb20d9-f12c-9963-2829-3c64d8d8cb14
3,a,0.5046331,1c22a468-9492-2173-9e4f-9003a23d02b7
4,c,0.6014692,7ea4d431-4dec-3017-3d13-cc9ef7f1c0ee
5,c,0.5000071,782c5346-f5f7-b90e-c686-8d41fa85233b
6,c,0.8392881,245f5516-0cb8-391a-e1e5-fadddc8e54ba
7,b,0.5938637,e30bab29-2df0-3fb0-535f-58d1e7bd83c0
8,a,0.765905,5e9cd21b-88c5-bbf5-7215-6409e115a2a4


In [19]:
proc.kill()